In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16911514
paper_name = 'kawahata_iefuji_2006' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [42]:
sheets = ['Resistant','Sensitive']

original_data_list = []
for ixs, s in enumerate(sheets):
    original_data = pd.read_excel('raw_data/hits.xlsx', sheet_name=s)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    
    original_data1 = original_data.iloc[:,[0,2,3,4]].copy()
    original_data1.columns = ['orf','lactic','acetic','hydrochloric']
    
    original_data2 = original_data.iloc[:,[5,7,8,9]].copy()
    original_data2.columns = ['orf','lactic','acetic','hydrochloric']
    
    original_data = pd.concat([original_data1, original_data2], axis=0)
    
    original_data['orf'] = original_data['orf'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data['orf'] = translate_sc(original_data['orf'].values, to='orf')
    
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    original_data = original_data.loc[t,:]
    
    original_data.set_index('orf', inplace=True)
    
    for c in original_data.columns:
        original_data[c] = original_data[c].apply(lambda x: 0 if str(x) == 'nan' else np.power(-1, ixs) * len(x))
    
    original_data = original_data.groupby(original_data.index).mean()
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 61 x 10
                                            ORF name  Gene Lactic acid  \
0  A plus sign indicates that the gene deletion s...   NaN         NaN   
1                                          Cell wall   NaN         NaN   
2                                            YLR286C  CTS1           +   
3                                            YHR143W  DSE2           +   
4                                            YNL327W  EGT2           +   

  Acetic acid Hydrochloric acid      ORF name.1 Gene.1 Lactic acid.1  \
0         NaN               NaN             NaN    NaN           NaN   
1         NaN               NaN  Other function    NaN           NaN   
2         NaN               NaN         YLR131C   ACE2             +   
3         NaN               NaN         YPL078C   ATP4             +   
4           +                 +         YJR092W   BUD4           NaN   

  Acetic acid.1 Hydrochloric acid.1  
0           NaN                 NaN  
1           

In [43]:
original_data = pd.concat(original_data_list, axis=1)

In [44]:
original_data.shape

(420, 6)

In [45]:
original_data = original_data.groupby(original_data.index).mean()

In [46]:
original_data.shape

(420, 6)

In [47]:
original_data.index.name = 'orf'

In [48]:
original_data[original_data.isnull()] = 0

In [49]:
original_data.head()

,lactic,acetic,hydrochloric,lactic,acetic,hydrochloric
orf,,,,,,
YAL014C,1.0,0.0,1.0,0.0,0.0,0.0
YAL016W,0.0,0.0,0.0,0.0,-1.0,-1.0
YAL021C,0.0,0.0,0.0,0.0,-1.0,-1.0
YAL026C,0.0,0.0,0.0,-1.0,-1.0,-1.0
YAL031C,0.0,0.0,1.0,0.0,0.0,0.0


# Prepare the final dataset

In [64]:
data = original_data.copy()

In [65]:
dataset_ids = [422, 420, 418, 177, 421, 419]
datasets = datasets.reindex(index=dataset_ids)

In [66]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [67]:
data.head()

dataset_id,422,420,418,177,421,419
data_type,value,value,value,value,value,value
orf,,,,,,
YAL014C,1.0,0.0,1.0,0.0,0.0,0.0
YAL016W,0.0,0.0,0.0,0.0,-1.0,-1.0
YAL021C,0.0,0.0,0.0,0.0,-1.0,-1.0
YAL026C,0.0,0.0,0.0,-1.0,-1.0,-1.0
YAL031C,0.0,0.0,1.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [68]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [69]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,422,420,418,177,421,419
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
12,YAL014C,1.0,0.0,1.0,0.0,0.0,0.0
14,YAL016W,0.0,0.0,0.0,0.0,-1.0,-1.0
19,YAL021C,0.0,0.0,0.0,0.0,-1.0,-1.0
24,YAL026C,0.0,0.0,0.0,-1.0,-1.0,-1.0
29,YAL031C,0.0,0.0,1.0,0.0,0.0,0.0


# Normalize

In [70]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [71]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [72]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,422,420,418,177,421,419,422,420,418,177,421,419
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
12,YAL014C,1.0,0.0,1.0,0.0,0.0,0.0,9.981505,0.0,9.671123,0.000000,0.000000,0.000000
14,YAL016W,0.0,0.0,0.0,0.0,-1.0,-1.0,0.000000,0.0,0.000000,0.000000,-4.606256,-5.146973
19,YAL021C,0.0,0.0,0.0,0.0,-1.0,-1.0,0.000000,0.0,0.000000,0.000000,-4.606256,-5.146973
24,YAL026C,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.000000,0.0,0.000000,-5.983702,-4.606256,-5.146973
29,YAL031C,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,9.671123,0.000000,0.000000,0.000000


# Print out

In [73]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [74]:
from IO.save_data_to_db3 import *

In [75]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/6 [00:00<?, ?it/s]

Deleting all datasets for PMID 16911514...
Inserting the new data...


100%|██████████| 6/6 [00:04<00:00,  1.47it/s]

Updating the data_modified_on field...
